# Chapter 8: Ray Tracing - Core Implementation

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/adiel2012/computer-vision/blob/main/chapter_08_ray_tracing.ipynb)

**Ray tracing** is a rendering technique that traces the path of light rays through a scene. Unlike rasterization, ray tracing naturally handles reflection, refraction, and shadows by simulating light physics.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import math
from typing import Tuple, List, Optional
from dataclasses import dataclass

In [ ]:
class Vec3:
    def __init__(self, x: float = 0.0, y: float = 0.0, z: float = 0.0):
        self.x, self.y, self.z = x, y, z
    
    def __add__(self, other):
        return Vec3(self.x + other.x, self.y + other.y, self.z + other.z)
    
    def __sub__(self, other):
        return Vec3(self.x - other.x, self.y - other.y, self.z - other.z)
    
    def __mul__(self, scalar: float):
        return Vec3(self.x * scalar, self.y * scalar, self.z * scalar)
    
    def __truediv__(self, scalar: float):
        return Vec3(self.x / scalar, self.y / scalar, self.z / scalar)
    
    def __neg__(self):
        return Vec3(-self.x, -self.y, -self.z)
    
    def dot(self, other) -> float:
        return self.x * other.x + self.y * other.y + self.z * other.z
    
    def cross(self, other):
        return Vec3(
            self.y * other.z - self.z * other.y,
            self.z * other.x - self.x * other.z,
            self.x * other.y - self.y * other.x
        )
    
    def length(self) -> float:
        return math.sqrt(self.dot(self))
    
    def normalize(self):
        l = self.length()
        return self / l if l > 0 else Vec3(0, 0, 0)
    
    def __repr__(self):
        return f"Vec3({self.x:.3f}, {self.y:.3f}, {self.z:.3f})"

class Color:
    def __init__(self, r: float = 0.0, g: float = 0.0, b: float = 0.0):
        self.r, self.g, self.b = r, g, b
    
    def __add__(self, other):
        return Color(self.r + other.r, self.g + other.g, self.b + other.b)
    
    def __mul__(self, scalar: float):
        if isinstance(scalar, (int, float)):
            return Color(self.r * scalar, self.g * scalar, self.b * scalar)
        # Color * Color (component-wise)
        return Color(self.r * scalar.r, self.g * scalar.g, self.b * scalar.b)
    
    def __rmul__(self, scalar: float):
        return self.__mul__(scalar)
    
    def clamp(self):
        return Color(
            max(0.0, min(1.0, self.r)),
            max(0.0, min(1.0, self.g)),
            max(0.0, min(1.0, self.b))
        )
    
    def to_rgb(self) -> Tuple[int, int, int]:
        c = self.clamp()
        return (int(c.r * 255), int(c.g * 255), int(c.b * 255))

print("✓ Base classes loaded")

## 1. Ray Representation

A ray is defined by:

$$
\mathbf{r}(t) = \mathbf{o} + t\mathbf{d}
$$

where:
- $\mathbf{o}$ = origin point
- $\mathbf{d}$ = direction vector (usually normalized)
- $t$ = parameter ($t \geq 0$ for forward rays)

### Ray Generation

For a camera at position $\mathbf{e}$ looking at a pixel position $\mathbf{p}$:

$$
\text{origin} = \mathbf{e}
$$
$$
\text{direction} = \text{normalize}(\mathbf{p} - \mathbf{e})
$$

In [ ]:
@dataclass
class Ray:
    origin: Vec3
    direction: Vec3  # Should be normalized
    
    def at(self, t: float) -> Vec3:
        """Get point along ray at parameter t"""
        return self.origin + self.direction * t

print("✓ Ray class loaded")

## 2. Ray-Sphere Intersection

### Sphere Equation

A sphere with center $\mathbf{c}$ and radius $r$:

$$
|\mathbf{p} - \mathbf{c}|^2 = r^2
$$

### Intersection Algorithm

Substitute ray equation $\mathbf{p} = \mathbf{o} + t\mathbf{d}$:

$$
|\mathbf{o} + t\mathbf{d} - \mathbf{c}|^2 = r^2
$$

Expand to quadratic equation:

$$
at^2 + bt + c = 0
$$

where:
- $a = \mathbf{d} \cdot \mathbf{d}$ (usually 1 if direction is normalized)
- $b = 2\mathbf{d} \cdot (\mathbf{o} - \mathbf{c})$
- $c = (\mathbf{o} - \mathbf{c}) \cdot (\mathbf{o} - \mathbf{c}) - r^2$

### Solution

Discriminant:
$$
\Delta = b^2 - 4ac
$$

- $\Delta < 0$: no intersection
- $\Delta = 0$: tangent (one intersection)
- $\Delta > 0$: two intersections

$$
t = \frac{-b \pm \sqrt{\Delta}}{2a}
$$

Take smaller positive $t$ for closest hit.

In [ ]:
@dataclass
class HitRecord:
    t: float
    point: Vec3
    normal: Vec3
    material: 'Material'

class Material:
    def __init__(self, color: Color, ambient: float = 0.1, diffuse: float = 0.7, 
                 specular: float = 0.2, shininess: float = 32.0, reflectivity: float = 0.0):
        self.color = color
        self.ambient = ambient
        self.diffuse = diffuse
        self.specular = specular
        self.shininess = shininess
        self.reflectivity = reflectivity

class Sphere:
    def __init__(self, center: Vec3, radius: float, material: Material):
        self.center = center
        self.radius = radius
        self.material = material
    
    def intersect(self, ray: Ray) -> Optional[HitRecord]:
        """Ray-sphere intersection using quadratic formula"""
        oc = ray.origin - self.center
        
        # Quadratic coefficients
        a = ray.direction.dot(ray.direction)
        b = 2.0 * oc.dot(ray.direction)
        c = oc.dot(oc) - self.radius * self.radius
        
        # Discriminant
        discriminant = b * b - 4 * a * c
        
        if discriminant < 0:
            return None  # No intersection
        
        # Find nearest intersection
        sqrt_d = math.sqrt(discriminant)
        t = (-b - sqrt_d) / (2.0 * a)
        
        if t < 0.001:  # Too close (avoid self-intersection)
            t = (-b + sqrt_d) / (2.0 * a)
            if t < 0.001:
                return None
        
        # Calculate hit point and normal
        point = ray.at(t)
        normal = (point - self.center).normalize()
        
        return HitRecord(t, point, normal, self.material)

print("✓ Sphere intersection loaded")

## 3. Ray-Plane Intersection

### Plane Equation

A plane can be defined by a point $\mathbf{p}_0$ and normal $\mathbf{n}$:

$$
\mathbf{n} \cdot (\mathbf{p} - \mathbf{p}_0) = 0
$$

Or in implicit form:
$$
\mathbf{n} \cdot \mathbf{p} + d = 0
$$

### Intersection

Substitute ray: $\mathbf{p} = \mathbf{o} + t\mathbf{d}$

$$
\mathbf{n} \cdot (\mathbf{o} + t\mathbf{d} - \mathbf{p}_0) = 0
$$

Solve for $t$:
$$
t = \frac{\mathbf{n} \cdot (\mathbf{p}_0 - \mathbf{o})}{\mathbf{n} \cdot \mathbf{d}}
$$

If $\mathbf{n} \cdot \mathbf{d} = 0$, ray is parallel to plane (no intersection).

In [ ]:
class Plane:
    def __init__(self, point: Vec3, normal: Vec3, material: Material):
        self.point = point
        self.normal = normal.normalize()
        self.material = material
    
    def intersect(self, ray: Ray) -> Optional[HitRecord]:
        """Ray-plane intersection"""
        denom = self.normal.dot(ray.direction)
        
        if abs(denom) < 1e-6:  # Ray parallel to plane
            return None
        
        t = (self.point - ray.origin).dot(self.normal) / denom
        
        if t < 0.001:  # Behind ray or too close
            return None
        
        point = ray.at(t)
        normal = self.normal if denom < 0 else -self.normal  # Face towards ray
        
        return HitRecord(t, point, normal, self.material)

print("✓ Plane intersection loaded")

## 4. Lighting Model (Phong Illumination)

### Phong Reflection Model

$$
I = I_a k_a + I_p \left[ k_d (\mathbf{n} \cdot \mathbf{l}) + k_s (\mathbf{r} \cdot \mathbf{v})^\alpha \right]
$$

where:
- $I_a$ = ambient light intensity
- $k_a$ = ambient coefficient
- $I_p$ = point light intensity
- $k_d$ = diffuse coefficient
- $k_s$ = specular coefficient
- $\mathbf{n}$ = surface normal
- $\mathbf{l}$ = light direction
- $\mathbf{r}$ = reflection direction: $\mathbf{r} = 2(\mathbf{n} \cdot \mathbf{l})\mathbf{n} - \mathbf{l}$
- $\mathbf{v}$ = view direction
- $\alpha$ = shininess exponent

In [ ]:
@dataclass
class Light:
    position: Vec3
    color: Color
    intensity: float = 1.0

def reflect(v: Vec3, n: Vec3) -> Vec3:
    """Reflect vector v about normal n"""
    return v - n * (2.0 * v.dot(n))

def phong_lighting(hit: HitRecord, view_dir: Vec3, lights: List[Light], 
                   ambient: Color = None) -> Color:
    """Calculate Phong illumination"""
    if ambient is None:
        ambient = Color(0.1, 0.1, 0.1)
    
    mat = hit.material
    
    # Ambient component
    color = ambient * mat.color * mat.ambient
    
    for light in lights:
        # Light direction
        light_dir = (light.position - hit.point).normalize()
        
        # Diffuse component
        n_dot_l = max(0.0, hit.normal.dot(light_dir))
        diffuse = light.color * mat.color * (mat.diffuse * n_dot_l * light.intensity)
        
        # Specular component (Phong)
        reflect_dir = reflect(-light_dir, hit.normal)
        r_dot_v = max(0.0, reflect_dir.dot(view_dir))
        spec_factor = math.pow(r_dot_v, mat.shininess)
        specular = light.color * (mat.specular * spec_factor * light.intensity)
        
        color = color + diffuse + specular
    
    return color.clamp()

print("✓ Phong lighting loaded")

## 5. Shadow Rays

To determine if a point is in shadow:

1. Cast a **shadow ray** from hit point to light source
2. If the ray hits any object before reaching the light, the point is in shadow
3. Use a small epsilon offset to avoid self-intersection

### Shadow Ray Equation

$$
\mathbf{r}_{\text{shadow}} = \mathbf{p} + \epsilon\mathbf{n} + t(\mathbf{l} - \mathbf{p}), \quad t \in [0, 1]
$$

In [ ]:
class Scene:
    def __init__(self):
        self.objects = []
        self.lights = []
        self.ambient_light = Color(0.1, 0.1, 0.1)
        self.background_color = Color(0.2, 0.3, 0.5)  # Sky blue
    
    def add_object(self, obj):
        self.objects.append(obj)
    
    def add_light(self, light: Light):
        self.lights.append(light)
    
    def intersect(self, ray: Ray) -> Optional[HitRecord]:
        """Find closest intersection"""
        closest_hit = None
        closest_t = float('inf')
        
        for obj in self.objects:
            hit = obj.intersect(ray)
            if hit and hit.t < closest_t:
                closest_t = hit.t
                closest_hit = hit
        
        return closest_hit
    
    def is_in_shadow(self, point: Vec3, light_pos: Vec3) -> bool:
        """Check if point is in shadow from light"""
        to_light = light_pos - point
        distance = to_light.length()
        light_dir = to_light.normalize()
        
        # Shadow ray with small offset
        shadow_ray = Ray(point + light_dir * 0.001, light_dir)
        
        # Check if anything blocks the light
        for obj in self.objects:
            hit = obj.intersect(shadow_ray)
            if hit and hit.t < distance:
                return True
        
        return False

print("✓ Scene class loaded")

## 6. Recursive Ray Tracing (Reflections)

For reflective surfaces:

$$
C_{\text{total}} = (1 - k_r) C_{\text{local}} + k_r C_{\text{reflected}}
$$

where:
- $k_r$ = reflectivity coefficient
- $C_{\text{local}}$ = local Phong illumination
- $C_{\text{reflected}}$ = color from reflected ray (recursive)

Limit recursion depth to prevent infinite recursion.

In [ ]:
def trace_ray(ray: Ray, scene: Scene, depth: int = 0, max_depth: int = 5) -> Color:
    """Recursive ray tracing with reflections"""
    if depth >= max_depth:
        return scene.background_color
    
    hit = scene.intersect(ray)
    
    if not hit:
        return scene.background_color
    
    # Calculate view direction
    view_dir = -ray.direction
    
    # Local illumination with shadows
    visible_lights = []
    for light in scene.lights:
        if not scene.is_in_shadow(hit.point, light.position):
            visible_lights.append(light)
    
    local_color = phong_lighting(hit, view_dir, visible_lights, scene.ambient_light)
    
    # Reflection
    reflectivity = hit.material.reflectivity
    if reflectivity > 0.0:
        reflect_dir = reflect(ray.direction, hit.normal)
        reflect_ray = Ray(hit.point + reflect_dir * 0.001, reflect_dir)
        reflect_color = trace_ray(reflect_ray, scene, depth + 1, max_depth)
        
        local_color = local_color * (1 - reflectivity) + reflect_color * reflectivity
    
    return local_color.clamp()

print("✓ Ray tracing function loaded")

## 7. Camera and Ray Generation

### Perspective Camera

For a camera with:
- Eye position $\mathbf{e}$
- Look-at point $\mathbf{c}$
- Up vector $\mathbf{u}$
- Field of view (FOV) angle $\theta$

Camera basis vectors:
$$
\mathbf{w} = \text{normalize}(\mathbf{e} - \mathbf{c}) \quad \text{(backward)}
$$
$$
\mathbf{u} = \text{normalize}(\mathbf{up} \times \mathbf{w}) \quad \text{(right)}
$$
$$
\mathbf{v} = \mathbf{w} \times \mathbf{u} \quad \text{(up)}
$$

For pixel $(i, j)$ in image of size $(width, height)$:
$$
u = \frac{2i - width}{width}
$$
$$
v = \frac{height - 2j}{height}
$$

Ray direction:
$$
\mathbf{d} = \text{normalize}(u \cdot \mathbf{u} + v \cdot \mathbf{v} - \mathbf{w})
$$

In [ ]:
class Camera:
    def __init__(self, eye: Vec3, look_at: Vec3, up: Vec3, fov: float, aspect_ratio: float):
        self.eye = eye
        
        # Build camera coordinate frame
        self.w = (eye - look_at).normalize()  # backward
        self.u = up.cross(self.w).normalize()  # right
        self.v = self.w.cross(self.u)  # up
        
        # Viewport dimensions
        theta = fov * math.pi / 180.0
        h = math.tan(theta / 2.0)
        viewport_height = 2.0 * h
        viewport_width = aspect_ratio * viewport_height
        
        self.horizontal = self.u * viewport_width
        self.vertical = self.v * viewport_height
        self.lower_left = self.eye - self.horizontal / 2.0 - self.vertical / 2.0 - self.w
    
    def get_ray(self, u: float, v: float) -> Ray:
        """Generate ray for normalized coordinates (u, v) in [0, 1]"""
        direction = (self.lower_left + self.horizontal * u + 
                    self.vertical * v - self.eye).normalize()
        return Ray(self.eye, direction)

print("✓ Camera class loaded")

## 8. Complete Ray Tracer Renderer

In [ ]:
def render(scene: Scene, camera: Camera, width: int, height: int) -> np.ndarray:
    """Render scene to image"""
    image = np.zeros((height, width, 3), dtype=np.uint8)
    
    for j in range(height):
        if j % 20 == 0:
            print(f"Rendering line {j}/{height}")
        
        for i in range(width):
            # Normalized coordinates [0, 1]
            u = i / (width - 1)
            v = (height - 1 - j) / (height - 1)  # Flip y
            
            ray = camera.get_ray(u, v)
            color = trace_ray(ray, scene)
            
            image[j, i] = color.to_rgb()
    
    return image

print("✓ Renderer loaded")

## Example 1: Simple Scene with Reflections

In [ ]:
# Create scene
scene = Scene()

# Materials
red_diffuse = Material(Color(0.8, 0.2, 0.2), diffuse=0.8, specular=0.3, reflectivity=0.0)
green_diffuse = Material(Color(0.2, 0.8, 0.2), diffuse=0.8, specular=0.3, reflectivity=0.0)
blue_mirror = Material(Color(0.7, 0.7, 1.0), diffuse=0.3, specular=0.7, 
                       shininess=64, reflectivity=0.6)
gray_floor = Material(Color(0.5, 0.5, 0.5), diffuse=0.7, specular=0.1, reflectivity=0.2)

# Add objects
scene.add_object(Sphere(Vec3(0, 0, -5), 1.0, blue_mirror))  # Center reflective sphere
scene.add_object(Sphere(Vec3(-2.5, 0, -5), 0.8, red_diffuse))  # Left red sphere
scene.add_object(Sphere(Vec3(2.5, 0, -5), 0.8, green_diffuse))  # Right green sphere
scene.add_object(Plane(Vec3(0, -1, 0), Vec3(0, 1, 0), gray_floor))  # Floor

# Add lights
scene.add_light(Light(Vec3(5, 5, 0), Color(1, 1, 1), intensity=0.8))
scene.add_light(Light(Vec3(-5, 3, -2), Color(1, 0.9, 0.8), intensity=0.4))

# Setup camera
camera = Camera(
    eye=Vec3(0, 1, 2),
    look_at=Vec3(0, 0, -5),
    up=Vec3(0, 1, 0),
    fov=60,
    aspect_ratio=16/9
)

# Render
width, height = 400, 225
print(f"Rendering {width}x{height} image...")
image = render(scene, camera, width, height)

# Display
plt.figure(figsize=(12, 7))
plt.imshow(image)
plt.title('Ray Traced Scene with Reflections and Shadows')
plt.axis('off')
plt.tight_layout()
plt.show()

print("Ray tracing complete!")

## Summary

**Core ray tracing** implements realistic rendering by simulating light transport:

### Key Concepts

1. **Ray Representation**: $\mathbf{r}(t) = \mathbf{o} + t\mathbf{d}$

2. **Intersection Tests**:
   - Sphere: solve quadratic equation
   - Plane: solve linear equation
   - Triangle: Möller-Trumbore algorithm

3. **Lighting**:
   - Phong illumination model
   - Ambient, diffuse, specular components
   - Shadow rays for visibility testing

4. **Reflections**:
   - Recursive ray tracing
   - Reflection direction: $\mathbf{r} = \mathbf{d} - 2(\mathbf{d} \cdot \mathbf{n})\mathbf{n}$
   - Depth limiting

5. **Camera Model**:
   - Perspective projection
   - Ray generation per pixel
   - Field of view control

### Advantages of Ray Tracing
- Natural reflection and refraction
- Accurate shadows
- Physically based light transport
- Easy to understand and implement

### Next Steps
- Chapter 9: Acceleration structures (BVH, k-d trees)
- Chapter 10: Distribution ray tracing (soft shadows, DOF)
- Chapter 11: Path tracing for global illumination
- Chapter 12: Physically based rendering (PBR)